In [1]:
import pandas as pd
import numpy as np
# from sklearn.impute import KNNImputer

# Load the dataset
df = pd.read_csv("D:/testing_loan_data.csv", header=0)
df.head()

D:\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3326: DtypeWarning: Columns (8) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,id,member_id,loan_amnt,term,int_rate,emp_length,home_ownership,annual_inc,desc,purpose,...,inq_last_6mths,mths_since_recent_inq,revol_util,total_bc_limit,mths_since_last_major_derog,tot_hi_cred_lim,tot_cur_bal,application_approved_flag,internal_score,bad_flag
0,20000001,22419852,10000,36 months,22.15%,8 years,RENT,37000.0,NaN,debt_consolidation,...,1,3.0,73.10%,16200,NaN,14877.170280,36809,1,131,NaN
1,20000002,22349118,1400,36 months,18.24%,6 years,RENT,41000.0,NaN,other,...,0,9.0,11.50%,4000,NaN,4097.304770,19536,1,19,NaN
2,20000003,22398818,7000,36 months,12.49%,3 years,RENT,68900.0,NaN,debt_consolidation,...,0,11.0,48.10%,11900,80.0,12688.495160,241465,1,92,NaN
3,20000004,22419015,18000,60 months,16.29%,9 years,MORTGAGE,41000.0,NaN,debt_consolidation,...,1,0.0,38.10%,7600,73.0,7908.799817,179757,1,235,NaN
4,20000005,22388614,12000,36 months,12.99%,10+ years,MORTGAGE,64000.0,NaN,home_improvement,...,0,NaN,57.90%,21000,NaN,19378.561060,31953,1,157,NaN


In [2]:
# Inspect the dataset structure 
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 102505 entries, 0 to 102504
Data columns (total 23 columns):
 #   Column                       Non-Null Count   Dtype  
---  ------                       --------------   -----  
 0   id                           102505 non-null  int64  
 1   member_id                    102505 non-null  int64  
 2   loan_amnt                    102505 non-null  int64  
 3   term                         102505 non-null  object 
 4   int_rate                     102505 non-null  object 
 5   emp_length                   97184 non-null   object 
 6   home_ownership               102505 non-null  object 
 7   annual_inc                   102505 non-null  float64
 8   desc                         15194 non-null   object 
 9   purpose                      102505 non-null  object 
 10  percent_bc_gt_75             101459 non-null  float64
 11  bc_util                      101463 non-null  float64
 12  dti                          102505 non-null  float64
 13 

In [3]:
# Function to count null values and calculate the percentage of nulls
def NullCheck(data):
    d = dict()
    for col in data.columns:
        d[col] = [data[col].isnull().sum(), round(data[col].isnull().mean()*100,3)]
    ddd = pd.DataFrame(d).T
    return ddd.rename(columns={0: "NullSum", 1: "NullMean%"})

NullCheck(df)

,NullSum,NullMean%
id,0.0,0.000
member_id,0.0,0.000
loan_amnt,0.0,0.000
term,0.0,0.000
int_rate,0.0,0.000
emp_length,5321.0,5.191
home_ownership,0.0,0.000
annual_inc,0.0,0.000
desc,87311.0,85.177
purpose,0.0,0.000


In [4]:
# Drop columns with more than the threshold percentage of NaN values
missing_percentage = round(df.isnull().mean() * 100, 2)
threshold = 50
df = df.drop(columns=missing_percentage[missing_percentage > threshold].index)

# Function to convert columns to numeric values after removing "%" symbols
def mycolconvertper (data, collst):
    for col in collst:
        df[col] = df[col].replace({'%': ''}, regex=True).astype(float)
    return df
# List of columns to convert
lst = ['int_rate','revol_util']
df = mycolconvertper(df, lst)

# Function to extract digits and convert to the specified type
def mycolconvert(data, coldic):
    for column, typ in coldic.items():
        if typ == int:
            data[column] = data[column].str.extract( r'(\d+)').round().astype(typ)
        else:
            data[column] = data[column].str.extract( r'(\d+)').astype(typ)      
    return df
# Column type conversions
dic = {'term': int, 'emp_length': float}
df = mycolconvert(df, dic)

# imputer = KNNImputer(n_neighbors=2)
# df['emp_length'] = imputer.fit_transform(df[['emp_length']])
# df['emp_length'] = df['emp_length'].round().astype(int)

# Function to impute missing values based on skewness
def myimpute(data, collst):
    for col in collst:
        if data[col].dtype in [np.float64, np.int64]:
            skewness = data[col].skew()
            if abs(skewness) > 0.5:
                # High skewness, use median
                print(f"Column '{col}' has high skewness ({skewness:.2f}). Imputing with median.")
                data[col].fillna(data[col].median(), inplace=True)
            else:
                # Low skewness, use mean
                print(f"Column '{col}' has low skewness ({skewness:.2f}). Imputing with mean.")
                data[col].fillna(data[col].mean(), inplace=True)
        else:
            print(f"Column '{col}' is not numerical, skipping imputation.")
    return df
# Identify columns with missing values
dfnch = NullCheck(df)
lst = list(dfnch[dfnch['NullMean%'] != 0].index)
df = myimpute(df, lst)

# Perform one-hot encoding on columns
df = pd.get_dummies(df, columns=['home_ownership'], drop_first=False)
df = pd.get_dummies(df, columns=['purpose'], drop_first=False)
# return the empty bad_flag column to df
df['bad_flag'] = np.NAN
# Display the first 5 rows to check the result
df.head()

Column 'emp_length' has low skewness (-0.25). Imputing with mean.
Column 'percent_bc_gt_75' has low skewness (0.00). Imputing with mean.
Column 'bc_util' has high skewness (-0.56). Imputing with median.
Column 'mths_since_recent_inq' has high skewness (0.87). Imputing with median.
Column 'revol_util' has low skewness (0.33). Imputing with mean.


,id,member_id,loan_amnt,term,int_rate,emp_length,annual_inc,percent_bc_gt_75,bc_util,dti,...,purpose_house,purpose_major_purchase,purpose_medical,purpose_moving,purpose_other,purpose_renewable_energy,purpose_small_business,purpose_vacation,purpose_wedding,bad_flag
0,20000001,22419852,10000,36,22.15,8.0,37000.0,80.0,83.0,28.51,...,0,0,0,0,0,0,0,0,0,NaN
1,20000002,22349118,1400,36,18.24,6.0,41000.0,0.0,0.0,26.58,...,0,0,0,0,1,0,0,0,0,NaN
2,20000003,22398818,7000,36,12.49,3.0,68900.0,60.0,75.9,6.60,...,0,0,0,0,0,0,0,0,0,NaN
3,20000004,22419015,18000,60,16.29,9.0,41000.0,33.3,61.1,20.61,...,0,0,0,0,0,0,0,0,0,NaN
4,20000005,22388614,12000,36,12.99,10.0,64000.0,75.0,67.0,24.61,...,0,0,0,0,0,0,0,0,0,NaN


In [5]:
# Check the data types after encoding
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 102505 entries, 0 to 102504
Data columns (total 35 columns):
 #   Column                      Non-Null Count   Dtype  
---  ------                      --------------   -----  
 0   id                          102505 non-null  int64  
 1   member_id                   102505 non-null  int64  
 2   loan_amnt                   102505 non-null  int64  
 3   term                        102505 non-null  int32  
 4   int_rate                    102505 non-null  float64
 5   emp_length                  102505 non-null  float64
 6   annual_inc                  102505 non-null  float64
 7   percent_bc_gt_75            102505 non-null  float64
 8   bc_util                     102505 non-null  float64
 9   dti                         102505 non-null  float64
 10  inq_last_6mths              102505 non-null  int64  
 11  mths_since_recent_inq       102505 non-null  float64
 12  revol_util                  102505 non-null  float64
 13  total_bc_limit

In [6]:
df.duplicated(subset=['id']).sum()

0

In [7]:
df.duplicated().sum()

0

In [8]:
# Save the DataFrame to a CSV file, excluding the index column
file_path = 'D:/cleaned_testing_loan_data.csv'
df.to_csv(file_path, index=False)  